In [1]:
"""
Model Result Processing
Author: Liam Megraw, RIT Envirionmental Science Technician
Date last edited: 11/9/2022
ESRI ArcGIS Pro Version 2.5.2

Description:
This code uses the output of the "priority_grid_cells.ipynb" code along
with other attributes to assign four levels of priority to invasive plant 
predictions from the RIT-developed computer vision model.  

Output:
The final output is a point layer with the attributes of species, panorama date,
prediction priority, probability, and decision criteria.

How to Use:
Once complete, these points should be uploaded and added to the AGOL dashboards 
for viewing and verification by community scientists and professionals.
"""

'\nModel Result Processing\nAuthor: Liam Megraw, RIT Envirionmental Science Technician\nDate last edited: 9/22/2022\nESRI ArcGIS Pro Version 2.5.2\n\nDescription:\nThis code uses the output of the "priority_grid_cells.ipynb" code along\nwith other attributes to assign four levels of priority to invasive plant \npredictions from the RIT-developed computer vision model.  \n\nOutput:\nThe final output is a point layer with the attributes of species, panorama date,\nprediction priority, probability, and decision criteria.\n\nHow to Use:\nOnce complete, these points should be uploaded and added to the AGOL dashboards \nfor viewing and verification by community scientists and professionals.\n'

In [2]:
"""
Pseudocode Overview

Define gdb location
Define inputs: grid cells, model predictions

Per species and threshold:
    Assign near distance to points

Assign one of four priority levels:
    If a point is in a priority grid cell, within the past 3 growing seasons, and an isolated prediction, the priority is 1
    If a point is in a priority grid cell but older than the past 3 growing seasons and is part of a cluster, the priority is 2
    If a point is NOT in a priority grid cell, within the past 3 growing seasons, and an isolated prediction, the priority is 3
    If a point is NOT in a priority grid cell, is older than the past 3 growing seasons, and is part of a cluster, the priority is 4
"""

'\nPseudocode Overview\n\nDefine gdb location\nDefine inputs: grid cells, model predictions\n\nAssign one of four priority levels:\n    If a point is in a priority grid cell and within the past 3 growing seasons, priority is 1\n    If a point is in a priority grid cell but older than the past 3 growing seasons, priority is 2\n    If a point is NOT in a priority grid cell and within the past 3 growing seasons, priority is 3\n    If a point is NOT in a priority grid cell and older than the past 3 growing seasons, priority is 4\n'

In [26]:
#----- Get and set WD to gdb -----
import os
import arcpy
from arcpy import env
arcpy.env.workspace = 'C://Users//ltm4654//Documents//ArcGIS//Projects//Final_Deployment//Final_Deployment.gdb'
print(os.getcwd())

C:\Users\ltm4654\Documents\ArcGIS\Projects\Final_Deployment


In [27]:
# Define inputs -----
priority_grid = "Priority_Grid_NYS" # This is the default and should only need to be changed if you changed the output name in the "priority_grid_cells.ipynb"
# Name of county or prediction set for use as a suffix for the output
setName = "SVI_Project_All"
model_pred_observed = "SVI_Project_presences_all" # Merged point set for all desired species and decision criteria

ref_year = 2022

# -----

# Create dictionary of long names
# Names used for filtering in ArcGIS Online
species_fullnames = {
    "phrag": "'Phragmites, Unspecified'", # extra sinlge quotes are intentional since these are used in a field calculation
    "knot": "'Knotweed, Unspecified'",
    "wp": "'Wild Parsnip'",
    "toh": "'Tree-of-Heaven (Ailanthus)'",
    "pl": "'Purple Loosestrife'"
}

# Extract only the keys to a list
species_names = species_fullnames.keys()

relevant_years = [("'"+str(ref_year)+"%'"), ("'"+str(ref_year-1)+"%'"), ("'"+str(ref_year-2)+"%'")]
#print(relevant_years)
#whereClause = ("date LIKE "+relevant_years[0]+" Or date LIKE "+relevant_years[1]+ " Or date LIKE "+relevant_years[2])
# This where clause selects records within the past 3 years from the year of reference that are also an isolated (non-contiguous) presence at each threshold
whereClause = ("model_near = -1 And (date LIKE "+relevant_years[0]+" Or date LIKE "+relevant_years[1]+ " Or date LIKE "+relevant_years[2]+")")
#print(whereClause)
invertWhereClause = ("model_near != -1 And (date NOT LIKE "+relevant_years[0]+" Or date NOT LIKE "+relevant_years[1]+ " Or date NOT LIKE "+relevant_years[2]+")")

In [28]:
# record start time
start = time.time()

op_criteria = ["recall","F1","precision"]
allSpeciesNear = list()
intermediate_nears = list()
tmpNears = list()

geometries = ['POINT','LINE','POLYGON']
# IDs that iMap assigns to the various species of interest
jurisdiction_ids = {
    "phrag": 1277,
    "wp": 1182,
    "pl": 1265,
    "toh": 1167,
    "knot": (1074, 1191, 1278, 1479) # Includes Japanese knotweed, giant knotweed, bohemian knotweed, and knotweed species unknown
}

for species in species_names:
    print(species)
    # For iMap near dist
    # Build query to select 4 knotweed IDs
    if species is "knot":
        # Set initial SQL query
        idClause = "jurisdiction_species_id = "+str(jurisdiction_ids["knot"][0])
        # Add more conditions to query
        for ID in jurisdiction_ids["knot"][1:]:
            idClause = idClause + " Or jurisdiction_species_id = " + str(ID)
    # Otherwise, just select the single species ID
    else:
        idClause = "jurisdiction_species_id = "+str(jurisdiction_ids[species])
    
    # Add near distance to both model data and imap data of the same species (and threshold, if applicable)
    print("***Model near")
    for op_criterion in op_criteria:
        print("******"+op_criterion)
        # Select records of just that threshold and species to identify clustered and isolated points #Common_Nam = 'Knotweed, Unspecified' And op_criteria = 'F1'
        sel =  arcpy.management.SelectLayerByAttribute(model_pred_observed, "NEW_SELECTION", "op_criteria = '"+op_criterion+"' And Common_Nam = "+species_fullnames[species])
        
        # Add near dist that feature
        arcpy.analysis.Near(sel, sel, "12.4 Meters") # 12.4 meters is the 95th percentile of panorama separation in the test area of Broome County, NY
        # Copy to new feature so each near distance doesn't get overwritten
        tmpNear = "tmpNear_"+species+"_"+op_criterion
        arcpy.management.CopyFeatures(sel, tmpNear)
        arcpy.management.AlterField(tmpNear, "NEAR_DIST", "model_near", "model_near")
        arcpy.management.DeleteField(tmpNear, "NEAR_FID")
        intermediate_nears.append(tmpNear) # For merging
        tmpNears.append(tmpNear) # For deleting
        print(intermediate_nears)
        # Clear selection for next run
        arcpy.management.SelectLayerByAttribute(model_pred_observed, "CLEAR_SELECTION")
        
    # Merge all criteria for the species
    perSpeciesNear = "tmpNear_"+species
    arcpy.management.Merge(intermediate_nears, perSpeciesNear)
    intermediate_nears = list() # empty list for next run
        
    print("***iMap near")
    # Add near dist to confirmed iMap records just for the current species
    evalFields = list()
    for geometry in geometries:
        print("******"+geometry)
        imap_sel = arcpy.management.SelectLayerByAttribute("PRESENCE_"+geometry, "NEW_SELECTION", idClause)
        arcpy.analysis.Near(perSpeciesNear, imap_sel)
        arcpy.management.DeleteField(perSpeciesNear, "NEAR_FID")
        evalField = "imap_near_"+geometry
        arcpy.management.AlterField(perSpeciesNear, "NEAR_DIST", evalField, evalField)
        evalFields.append("!"+evalField+"!")
        arcpy.management.SelectLayerByAttribute("PRESENCE_"+geometry, "CLEAR_SELECTION")
    
    allSpeciesNear.append(perSpeciesNear) # for merging and deleting later
    
    # Add & calculate new field to identify minimum distance to any record geometry type
    arcpy.management.AddField(perSpeciesNear, "imap_near", "FLOAT")
    arcpy.management.CalculateField(perSpeciesNear, "imap_near", "min(["+evalFields[0]+","+evalFields[1]+","+evalFields[2]+"])")   
    # Delete fields that are no longer necessary
    deleteFields = list()
    for field in evalFields:
        deleteFields.append(field[1:-1]) # Return field without '!'
    arcpy.management.DeleteField(perSpeciesNear, deleteFields)
    
# Merge into final dataset for prioritization
mppNear = 'mppNear_'+setName
arcpy.management.Merge(allSpeciesNear, mppNear)
    
# Delete temp vars
del sel, imap_sel, tmpNear, evalField, evalFields, perSpeciesNear

print("Deleting temporary files...")
# Delete temporary features
# This way is necessary to delete the feature itself and not just its contents
import os
cws = arcpy.env.workspace
tempFeatures = tmpNears + allSpeciesNear
for layer in tempFeatures:
    input_path = os.path.join(cws, layer)
    if arcpy.Exists(input_path):
        arcpy.Delete_management(input_path)
del layer

print("Done!")
 
# record end time
end = time.time()
 
# print the difference between start
# and end time in milli. secs
print("Execution time :",
      (end-start), "s")

phrag
***Model near
******recall
['tmpNear_phrag_recall']
******F1
['tmpNear_phrag_recall', 'tmpNear_phrag_F1']
******precision
['tmpNear_phrag_recall', 'tmpNear_phrag_F1', 'tmpNear_phrag_precision']
***iMap near
******POINT
******LINE
******POLYGON
knot
***Model near
******recall
['tmpNear_knot_recall']
******F1
['tmpNear_knot_recall', 'tmpNear_knot_F1']
******precision
['tmpNear_knot_recall', 'tmpNear_knot_F1', 'tmpNear_knot_precision']
***iMap near
******POINT
******LINE
******POLYGON
wp
***Model near
******recall
['tmpNear_wp_recall']
******F1
['tmpNear_wp_recall', 'tmpNear_wp_F1']
******precision
['tmpNear_wp_recall', 'tmpNear_wp_F1', 'tmpNear_wp_precision']
***iMap near
******POINT
******LINE
******POLYGON
toh
***Model near
******recall
['tmpNear_toh_recall']
******F1
['tmpNear_toh_recall', 'tmpNear_toh_F1']
******precision
['tmpNear_toh_recall', 'tmpNear_toh_F1', 'tmpNear_toh_precision']
***iMap near
******POINT
******LINE
******POLYGON
pl
***Model near
******recall
['tmpNear_pl

In [19]:
# record start time
start = time.time()

arcpy.env.overwriteOutput = True

# Select points within grid cells (priority 1 and 2)
print("Assigning priority 1 and 2...")
p1_2 = arcpy.management.SelectLayerByLocation(mppNear, "INTERSECT", priority_grid)
# Copy to new feature
arcpy.management.CopyFeatures(p1_2, 'tmp_p1_2')

# Assign priority 1 (points within the past three growing seasons AND are isolated)
p1 = arcpy.management.SelectLayerByAttribute('tmp_p1_2', "SUBSET_SELECTION", whereClause)
arcpy.management.CalculateField('tmp_p1_2', "Priority_L", "1")

# Assign priority 2 (points that are *not* within the past three growing seasons that are also part of a cluster)
p2 = arcpy.management.SelectLayerByAttribute('tmp_p1_2', "SWITCH_SELECTION")
arcpy.management.CalculateField('tmp_p1_2', "Priority_L", "2")

# Clear selection for merging later
arcpy.management.SelectLayerByAttribute('tmp_p1_2', "CLEAR_SELECTION")

print("Assigning priority 3 and 4...")
# Select points not within grid cells (priority 3 and 4)
p3_4 = arcpy.management.SelectLayerByLocation(mppNear, "INTERSECT", priority_grid, "", "", "INVERT") # May need to use "does not intersect" instead
# Copy to new feature
arcpy.management.CopyFeatures(p1_2, 'tmp_p3_4')

# Assign priority 3 (those in the past three growing seasons AND are isolated)
p3 = arcpy.management.SelectLayerByAttribute('tmp_p3_4', "SUBSET_SELECTION", whereClause)
arcpy.management.CalculateField('tmp_p3_4', "Priority_L", "3")

# Assign priority 4
p4 = arcpy.management.SelectLayerByAttribute(p3, "SWITCH_SELECTION")
arcpy.management.CalculateField('tmp_p3_4', "Priority_L", "4")

# Clear selection for merging later
arcpy.management.SelectLayerByAttribute('tmp_p3_4', "CLEAR_SELECTION")


# Merge outputs
priority_sets = ["tmp_p1_2","tmp_p3_4"]
print("Merging results together...")
prioritizedName = "mpp_prioritized_"+setName
arcpy.management.Merge(priority_sets, prioritizedName)


# Delete temporary files
print("Deleting temporary files...")

import os
cws = arcpy.env.workspace
for layer in priority_sets:
    arcpy.management.Delete(layer)
    # This way is necessary to delete the feature itself and not just its contents
    input_path = os.path.join(cws, layer)
    if arcpy.Exists(input_path):
        arcpy.Delete_management(input_path)

print("Final output file named "+prioritizedName+" is ready for "+setName)

# record end time
end = time.time()
 
# print the difference between start
# and end time in milli. secs
print("Execution time :",
      (end-start), "s")

Assigning priority 1 and 2...
Assigning priority 3 and 4...
Merging results together...
Deleting temporary files...
Final output file named mpp_prioritized_HP_counties is ready for HP_counties
Execution time : 160.90822744369507 s


In [22]:
# Delete variables
# Initializing d with dir()
# This will store a list of all the variables in the program
d = dir()

# Check for user-defined variables in the directory
for obj in d:
    #checking for built-in variables/functions
    if not obj.startswith('__'):
        #deleting the said obj, since a user-defined function
        del globals()[obj]